In [2]:
import hardware.awg.AWG_M8190A_qudi as awg
#awg = AWG_M8190A_qudi
import hardware.awg.multi_channel_awg_seq as mcas

awg.awg._ch[1].sample_marker_low = 0
awg.awg._ch[1].sample_marker_high = 1.5
awg.awg._ch[1].output_amplitude = 0.5

Traceback (most recent call last):
  File "C:\src\qudi\logic\jupyterkernel\qzmqkernel.py", line 679, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "<ipython-input-2-c2346d4d5c63>", line 5, in <module>
    awg.awg._ch[1].sample_marker_low = 0
AttributeError: module 'hardware.awg.AWG_M8190A_qudi' has no attribute 'awg'


In [63]:
awg.awg._ch[1].sync_marker_high=1.5

In [64]:
awg.awg._ch[1].sample_marker_low = 0
awg.awg._ch[1].sample_marker_high = 1.5
AWG_M8190A_qudi.awg._ch[1].output_amplitude = 0.5


seq = mcas.MultiChSeq('Laser', ch_dict = {'awg':[1,2]})
seq.awg =awg
seq.start_new_segment('Starting segment')


seq.asc(length_mus = 10, green=True, orange = True)


seq.sequences['awg'][1].print_info()

0     Laser             10.000000
   0     Starting segment  10.000000
      0                       10.000000wait    1       0       
      1     _missing_smpls_   0.000000wait    0       0       


In [65]:
seq.write_seq()

writing sequence wait on AWG_M8190A_qudi ch 1 took 0.0130 seconds
writing sequence wait on AWG_M8190A_qudi ch 2 took 0.0100 seconds
MCAS 'Laser' has been generated.


In [93]:
seq.run_sequence()

writing sequence Laser on AWG_M8190A_qudi ch 1 took 0.0260 seconds
writing sequence Laser on AWG_M8190A_qudi ch 2 took 0.0140 seconds


In [69]:
seq.stop_awgs()

writing sequence wait on AWG_M8190A_qudi ch 1 took 0.0100 seconds
writing sequence wait on AWG_M8190A_qudi ch 2 took 0.0120 seconds


## Hahn echo

In [108]:
[1,400e-3]
0.5,200e-3

(0.5, 0.2)

In [109]:
AWG_M8190A_qudi.awg._ch[1].output_amplitude

0.5

In [110]:
awg = AWG_M8190A_qudi
import hardware.awg.multi_channel_awg_seq as mcas

awg.awg._ch[1].sample_marker_low = 0
awg.awg._ch[1].sample_marker_high = 1.5
AWG_M8190A_qudi.awg._ch[1].output_amplitude = 0.5

def awg_hahn_echo(taus):
    seq = mcas.MultiChSeq('hahn_echo', ch_dict={'awg': [1, 2]})
    phases = [0.,180.]
    awg_freq = 2206.54 # in MHz
    electron_amp = 0.2 # AU 300mV peak to peak
    electron_pi = 1.352/2 # us
    oscReadoutPhaseFreq = 0
    nitrogen_transition = 14.0733
    rf_safety = 1
    
    for tau in taus:
        for phase in phases:

            seq.start_new_segment('hahn')
            seq.asc(pdawg1=dict(type='sine', frequencies=[awg_freq], amplitudes=[electron_amp]),
                    length_mus=electron_pi / 2.)
            seq = awg_AddWait(seq, tau / 2.)
            seq.asc(pdawg1=dict(type='sine', frequencies=[awg_freq], amplitudes=[electron_amp]),
                    length_mus=electron_pi)
            seq = awg_AddWait(seq, tau / 2.)
            seq.asc(pdawg1=dict(type='sine', frequencies=[awg_freq], amplitudes=[electron_amp],
                                phases=[phase]),
                    length_mus=electron_pi / 2.)
            seq.asc(length_mus=0.1)
            
#             seq = AddRfSafety(seq)
            seq = awg_AddReadout(seq)
    return seq

# def awg_AddReadout(seq):
#     length_laserreadout_ns = 3000
#     length_lasersafety_ns = 1000
#     length_laserreadout_smpl = length_laserreadout_ns / 1e3 * 12e3
    
#     while length_laserreadout_smpl % 64 != 0:
#         length_laserreadout_smpl += 1
        
#     seq.asc(length_mus=0.5)  # 2nd step
#     seq.asc(timetagger=True,green=True,orange=True, length_smpl=length_laserreadout_smpl)
    
#     seq.asc(length_mus= length_lasersafety_ns / 1e3)
#     return seq

def awg_AddReadout(seq):
    length_laserreadout_ns = 3000
    length_lasersafety_ns = 4000
    AOMdelay_ns = 600
    length_laserreadout_smpl = length_laserreadout_ns / 1e3 * 12e3
    AOMdelay_sample = AOMdelay_ns  / 1e3 * 12e3
    
    while length_laserreadout_smpl % 64 != 0:
        length_laserreadout_smpl += 1
        
    seq.asc(length_mus=0.5)  # 2nd step
#     seq.asc(length_mus = 10, green=False, orange = True) #green on, AOM off

    seq.asc(timetagger=False,green=True,orange=False, length_smpl=AOMdelay_sample)
    seq.asc(timetagger=True,green=True,orange=True, length_smpl=length_laserreadout_smpl)    
    seq.asc(timetagger=True,green=False,orange=True, length_smpl=AOMdelay_sample)

    seq.asc(length_mus= length_lasersafety_ns / 1e3)
    return seq





def awg_AddWait(seq, time, orange=False, green=False):
    enable_reuse_segments = False
    if time < 0.:
        for i in range(3): 
            print ('Negative time for sequence in AWG!!!')
    elif time > 10.:
        if int(time / (32 / 12.)) >= 1:
            seq.start_new_segment('wait_loop', reuse_segment=enable_reuse_segments,
                                  loop_count=int(time / (32 / 12.)))
            seq.asc(length_smpl=32000, orange=orange, green=green)
        seq.start_new_segment('wait_additional')
        seq.asc(length_mus=time % (32 / 12.), orange=orange, green=green)
    else:
        seq.asc(length_mus=time, orange=orange, green=green)
    return seq

def AddRfSafety(seq):
    rf_safety = 1
    seq.asc(length_mus=rf_safety)
    return seq

# taus = np.linspace(100,1000,91)
# taus = np.arange(0.01,130,1.)
taus = np.arange(1., 50, .4)


seqEcho = awg_hahn_echo(taus)
seqEcho.awg = awg




# pulsedmasterlogic.set_measurement_settings(invoke_settings=False, 
#                                            controlled_variable= taus,
#                                            number_of_lasers=2*len(taus), 
#                                            laser_ignore_list=[], 
#                                            alternating=True, 
#                                            units=('s', 'arb. u.'))




In [102]:

pulsedmasterlogic.set_measurement_settings(invoke_settings=False, 
                                           controlled_variable= taus*1e-6,
                                           number_of_lasers=2*len(taus), 
                                           laser_ignore_list=[], 
                                           alternating=True, 
                                           units=('s', 'arb. u.'))

In [103]:
len(taus)

123

In [104]:
# seqEcho.sequences['awg'][1].print_info()

In [105]:
seqEcho.write_seq()

MCAS 'hahn_echo' has been generated.


In [106]:
seqEcho.run_sequence()

writing sequence hahn_echo on AWG_M8190A_qudi ch 1 took 6.7611 seconds
writing sequence hahn_echo on AWG_M8190A_qudi ch 2 took 6.7900 seconds


In [107]:
seqEcho.stop_awgs()

writing sequence wait on AWG_M8190A_qudi ch 1 took 0.0170 seconds
writing sequence wait on AWG_M8190A_qudi ch 2 took 0.0180 seconds


33.333333333333336